# PARTIE 1

# importer la dataframe 

In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# chemin de la data
path =r'C:\Users\utilisateur\Desktop\Run_Questionnaire\DataSet'
filenames = glob.glob(path + "/DataSet*.csv")
# une boucle
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# fusion des dataframe
dataset_brut = pd.concat(dfs, ignore_index=True)

ValueError: No objects to concatenate

In [2]:
#visualisations
dataset_brut

NameError: name 'dataset_brut' is not defined

In [ ]:
# verif le % de missing
for col in dataset_brut.columns:
    pct_missing = np.mean(dataset_brut[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

# lisser majuscule et minuscules et chiffres

In [ ]:
#on uniformise majuscule et miniscule
dataset1=dataset_brut.iloc[:,:5].replace({'A':'a', 'B': 'b', 'C': 'c'})

In [ ]:
#on enleve tout ce qui n'est pas a b ou c
dataset1=dataset1[dataset1.isin(["a","b","c"])]
#dataset1= dataset1.fillna(np.NaN) #pas necessaire car on a crée une nouvelle instance

In [ ]:
#selection de la deuxieme partie
dataset2=dataset_brut.iloc[:,5:10]

In [ ]:
#on enleve tout ce qui n'est pas 1 2 ou 3
dataset2=dataset2[dataset2.isin(['1','1.0',1.,'2','2.0',2.,'3','3.0',3.])]

In [ ]:
#lissage des chiffres
dataset2=dataset2.replace({'3':3.0, '2': 2.0, '1': 1.0})

In [ ]:
#on regroupe tout tout pour obtenir un dataset pret pour la prochaine étape
dataset = pd.concat([dataset1, dataset2, dataset_brut['Interpretation']], axis=1)

# on enlève les lignes comprenant des valeures vides Na

In [ ]:
dataset=dataset.dropna()

# attribution des valeurs de calcul

In [ ]:
# attribution des points pour a b et c 1 2 et 3
#attention a l'ordre ou l'on le fait pour ne pas remplacer 2 par 0 apres avoir remplacé c par 2
dataset=dataset.replace({1:1, 2: 0, 3: 2})
dataset=dataset.replace({'a':1.0, 'b': 0.0, 'c': 2.0})

In [ ]:
dataset

# divisier X et y

In [ ]:
# assigner X à toutes les questions et y à la classe
# on a exlus exclus le score total un peut plus haut en concatenant
X = dataset.iloc[:, :10].values
y = dataset.iloc[:,10:11].values

# PARTIE 2

# Facultatif : KNN from scratch

Préparez une fonction permettant de calculer les 3 différentes distances : Euclidean, Manhattan et Minkowski, (def distance(metric=’ Euclidean’, **kargs)).

In [ ]:
#splitting data in test and train segments
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
#Creating a helper function
def takeSecond(elem):
    return elem[1]

In [ ]:
def KNNClassify(X_test, Y_train = y_train,X_train = X_train, k = 10):
   
    min_dist = []
    #for every example in the training set, calculate eucledien distance against the test example
    for i,point in enumerate(X_train):
        d1 = (point[0]-X_test[0])**2
        d2 = (point[1]-X_test[1])**2
        d3 = (point[2]-X_test[2])**2
        d4 = (point[3]-X_test[3])**2
        dist = np.sqrt(d1+d2+d3+d4)
        #append the calculated distance in a list
        min_dist.append((i,dist))
    #sort distances in ascending order    
    min_dist.sort(key = takeSecond)
    
    #get top k nearest neighbours
    neighbours = min_dist[:k]
    #get index of the minimum distances
    idx = []
    for tup in neighbours:
        idx.append(tup[0])
    #check which label has majority
    output = Y_train[idx]
    values, counts = np.unique(output, return_counts=True)
    #return label with majority occurence
    max_idx = np.argmax(counts)
    return values[max_idx]

In [ ]:
predictions = list(map(KNNClassify, X_test))

In [ ]:
def accuracy(pred , y_test):
    count = 0
    for i in range(len(pred)):
        if pred[i] == y_test[i]:
            count +=1
            
    return print("Accuracy =", (count/len(pred))*100, "%")

In [ ]:
accuracy(predictions, y_test)

# PARTIE 2
# KNN avec scikit learn

on va plutot utiliser le KFold qui sera plus precis

In [ ]:
# ici on va utiliser le Kfold 
import numpy as np
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=7)


performance = []

kf = KFold(n_splits=7)
for train, test in kf.split(X):
    X_train = X[train,:]
    y_train = y[train,:]
    
    X_test = X[test,:]
    y_test = y[test,:]
    
    
    model.fit(X_train,y_train)
    Per = model.predict(X_test)
    
    performance.append(accuracy_score(y_test, Per))
    
print(performance)

scaling inutile ici car nos variables ne varient pas assez 

ici on entraine le modèle grace a l'aide de KNeighborsClassifier

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=8)
classifier.fit(X_train, y_train)

on fait une prediction

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
errors = []
for k in range(2,15):
    knn = neighbors.KNeighborsClassifier(k)
    errors.append(100*(1 - knn.fit(xtrain, ytrain).score(xtest, ytest)))
plt.plot(range(2,15), errors, 'o-')
plt.show()

on peut enfin afficher les resultats comme ceci

In [ ]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2*100,"%")

# Partie 3 : JObLIB

In [ ]:
import joblib

In [ ]:
filename = 'knnfinal.sav'
joblib.dump(classifier, filename)